<a href="https://colab.research.google.com/github/Praveen76/Loan-Approval-Classifier-Model-using-Non-Linear-Models/blob/main/Loan_Approval_Classifier_FinalPipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [266]:
# !git clone https://github.com/Praveen76/Loan-Approval-Classifier-Model-using-Non-Linear-Models.git
# %cd Loan-Approval-Classifier-Model-using-Non-Linear-Models

In [267]:
# %pip install shap xgboost plotly scikit-plot seaborn statsmodels tabulate matplotlib imblearn nbformat>=4.2.0


In [268]:

import sys

# Print Python version
print("Python version")
print(sys.version)
print("")

# Print installed package versions
print("Installed package versions")
for module in sys.modules:
    try:
        version = sys.modules[module].__version__
        print(f"{module}: {version}")
    except AttributeError:
        pass


Python version
3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]

Installed package versions
re: 2.2.1
ipykernel._version: 5.5.6
json: 2.0.9
IPython.core.release: 7.34.0
logging: 0.5.1.2
zlib: 1.0
traitlets._version: 5.7.1
traitlets: 5.7.1
socketserver: 0.4
argparse: 1.1
platform: 1.0.8
pygments: 2.16.1
ptyprocess: 0.7.0
pexpect: 4.9.0
decorator: 4.4.2
pickleshare: 0.7.5
backcall: 0.2.0
wcwidth: 0.2.13
prompt_toolkit: 3.0.43
urllib.request: 3.10
IPython: 7.34.0
ipython_genutils._version: 0.2.0
ipython_genutils: 0.2.0
jupyter_client._version: 6.1.12
_ctypes: 1.1.0
ctypes: 1.1.0
zmq.sugar.version: 23.2.1
zmq.sugar: 23.2.1
zmq: 23.2.1
jupyter_core.version: 5.7.1
jupyter_core: 5.7.1
platformdirs.version: 4.2.0
platformdirs: 4.2.0
jupyter_client: 6.1.12
ipykernel: 5.5.6
_curses: b'2.2'
dateutil: 2.8.2
six: 1.16.0
_decimal: 1.70
decimal: 1.70
setuptools._distutils: 3.10.12
_csv: 1.0
csv: 1.0
setuptools.version: 67.7.2
setuptools._vendor.packaging: 23.0
setuptools.extern.packaging: 23.0
setu

In [269]:
import pandas as pd
import numpy as np
from scipy import stats

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

import matplotlib.pyplot as plt
import seaborn as sns

import scipy.stats as stats
from statsmodels.formula.api import ols
import statsmodels.api as sm

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import recall_score, precision_score, f1_score, accuracy_score
import sklearn.metrics as metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from tabulate import tabulate

import warnings

warnings.filterwarnings("ignore")

In [270]:
loan_original=pd.read_csv('./data.csv',parse_dates=['DateAdded','ApprovalDate'])
loan_original=loan_original.infer_objects()

loan_original.head()

,LeadID,DateAdded,LeadSourceGroup,LoanOfficer,Group,ZipCode,LoanNumber,ApprovalDate,BranchCode,LoanPurpose,...,BorrowerAge,BorrowerYearsInSchool,BorrowerTotalMonthlyIncome,BorrowerOwnRent,BorrowerCity,BorrowerState,IsCoBorrowerower,CoBorrowerYearsInSchool,CoBorrowerAge,CoBorrowerTotalMonthlyIncome
0,13067,2013-05-01 18:54:00,NaN,"Narmour, Derrick",Refinance Team - 4103,75078.0,41032026021,NaT,4103.0,Refinance ...,...,36,18,6083.00,NaN,Prosper,TX,True,0,0,0.0
1,14396,2013-05-07 16:53:00,TV,"Johnson, Adrienne",Refinance Team - 4103,76064.0,4103198104,2019-07-21,4103.0,Refinance Cash-out ...,...,52,16,8883.71,Own,Maypearl,TX,True,12,55,0.0
2,14855,2013-05-09 12:53:00,NaN,"Frank, Michael",Refinance Team - 4103,76017.0,4102192789,2019-03-26,4102.0,Refinance Cash-out ...,...,60,12,17538.30,Own,Arlington,TX,True,0,0,0.0
3,26269,2013-07-01 17:25:00,TV,"Benitez, Bryan",Refinance Team - 4102,75238.0,4102193097,2019-04-05,4102.0,Refinance Cash-out ...,...,54,16,13299.09,Own,Dallas,TX,True,0,0,0.0
4,37010,2013-08-30 09:58:00,TV,"Nguyen, Tu",Refinance Team - 4102,77450.0,4102190764,2019-02-01,4102.0,Refinance Cash-out ...,...,49,16,6254.91,Own,Katy,TX,True,0,0,0.0


In [271]:
loan_original.columns

Index(['LeadID', 'DateAdded', 'LeadSourceGroup', 'LoanOfficer', 'Group',
       'ZipCode', 'LoanNumber', 'ApprovalDate', 'BranchCode', 'LoanPurpose',
       'LoanType', 'TotalLoanAmount', 'CreditScore', 'CLTV', 'DTI',
       'BorrowerAge', 'BorrowerYearsInSchool', 'BorrowerTotalMonthlyIncome',
       'BorrowerOwnRent', 'BorrowerCity', 'BorrowerState', 'IsCoBorrowerower',
       'CoBorrowerYearsInSchool', 'CoBorrowerAge',
       'CoBorrowerTotalMonthlyIncome'],
      dtype='object')

In [272]:

def data_sanity_check(df: pd.DataFrame):

  # Remove duplicate records.
  df.drop_duplicates(subset='LoanNumber', keep='first', inplace=True)

  # Filter out wrong BorrowerYearsInSchool records
  df = df[df['BorrowerYearsInSchool']>=1]

  #Strip off white space
  loan_original.LoanPurpose=[entry.strip() for entry in loan_original.LoanPurpose]
  loan_original.LoanType=[entry.strip() for entry in loan_original.LoanType]

  df.reset_index(drop=True, inplace=True)

  return df

loan_original = data_sanity_check(loan_original)




In [273]:
def data_transformation(df: pd.DataFrame):

  # Create Approved column
  df['Approved'] = [1 if not pd.isna(date) else 0 for date in df['ApprovalDate']]


  #Recategorize Loan Purpose column
  LP = ['Refinance','Purchase']
  df['LoanPurpose']= [v if v in LP else 'Refinance' for v in df.LoanPurpose]

  #Calculate difference between current date and DateAdded in days
  df['Diff']=pd.Timestamp.now().normalize()-df['DateAdded'][df.ApprovalDate.isna()]
  df['Diff']=[entry.days for entry in df['Diff']]

  df['Approved']=[0 if ((x>=90) & (y=='Purchase')) else 1 for (x,y) in zip(df.Diff, df.LoanPurpose)]
  df['Approved']=[0 if ((x>=45) & (y=='Refinance')) else 1 for (x,y) in zip(df.Diff, df.LoanPurpose)]

  #
  df['Approved']=df['Approved'].astype('category')
  df['IsCoBorrowerower']=df['IsCoBorrowerower'].astype('category')

  df.drop('Diff', axis=1, inplace=True)

  LP = ['Own','Rent']
  df['BorrowerOwnRent']= [v if v in LP else 'Own' for v in df.BorrowerOwnRent]
  df['BorrowerOwnRent'][df['BorrowerOwnRent'].isna()]=df['BorrowerOwnRent'].mode().values

  #Add Borrower & Co-Borrower's income
  df['TotalIncome']=df.BorrowerTotalMonthlyIncome + df.CoBorrowerTotalMonthlyIncome

  #Regroup LeadSourceGroup
  LSR = ['Internet', 'TV','Radio','Repeat Client' ]
  df['LeadSourceGroup'] = [v if v in LSR else 'Other' for v in df.LeadSourceGroup]

  zips = ['75', '76', '77', '78', '79']
  df['ZipCode'] = [str(zp)[:2] for zp in df['ZipCode']]
  df['ZipCode'] = [v if v in zips else 'Other' for v in df.ZipCode]

  #Create bins for Education
  # Assuming 'BorrowerYearsInSchool' is a column in the DataFrame
  bins = [0, 12, 16, 18, df['BorrowerYearsInSchool'].max()]
  group = ['Higher School', 'UnderGrad', 'PostGrad', 'PHD']
  df['Education'] = pd.cut(df['BorrowerYearsInSchool'], bins, labels=group)

  return df


loan_original = data_transformation(loan_original)

In [274]:
def filter_extreme_vals(df: pd.DataFrame):

  ## Filter out extreme outliers
  df=df[df.CLTV<110]
  df=df[df.TotalLoanAmount<600000]
  df=df[df.CreditScore>550]
  df=df[df.TotalIncome<=30000]

  return df

loan_original = filter_extreme_vals(loan_original)

In [275]:
loan_original.head()

,LeadID,DateAdded,LeadSourceGroup,LoanOfficer,Group,ZipCode,LoanNumber,ApprovalDate,BranchCode,LoanPurpose,...,BorrowerOwnRent,BorrowerCity,BorrowerState,IsCoBorrowerower,CoBorrowerYearsInSchool,CoBorrowerAge,CoBorrowerTotalMonthlyIncome,Approved,TotalIncome,Education
0,13067,2013-05-01 18:54:00,Other,"Narmour, Derrick",Refinance Team - 4103,75,41032026021,NaT,4103.0,Refinance,...,Own,Prosper,TX,True,0,0,0.0,0,6083.00,PostGrad
1,14396,2013-05-07 16:53:00,TV,"Johnson, Adrienne",Refinance Team - 4103,76,4103198104,2019-07-21,4103.0,Refinance,...,Own,Maypearl,TX,True,12,55,0.0,1,8883.71,UnderGrad
2,14855,2013-05-09 12:53:00,Other,"Frank, Michael",Refinance Team - 4103,76,4102192789,2019-03-26,4102.0,Refinance,...,Own,Arlington,TX,True,0,0,0.0,1,17538.30,Higher School
3,26269,2013-07-01 17:25:00,TV,"Benitez, Bryan",Refinance Team - 4102,75,4102193097,2019-04-05,4102.0,Refinance,...,Own,Dallas,TX,True,0,0,0.0,1,13299.09,UnderGrad
4,37010,2013-08-30 09:58:00,TV,"Nguyen, Tu",Refinance Team - 4102,77,4102190764,2019-02-01,4102.0,Refinance,...,Own,Katy,TX,True,0,0,0.0,1,6254.91,UnderGrad


In [276]:
loan_original.isna().sum()

LeadID                             0
DateAdded                          0
LeadSourceGroup                    0
LoanOfficer                        4
Group                              4
ZipCode                            0
LoanNumber                         0
ApprovalDate                    3609
BranchCode                         2
LoanPurpose                        0
LoanType                           0
TotalLoanAmount                    0
CreditScore                        0
CLTV                               0
DTI                              215
BorrowerAge                        0
BorrowerYearsInSchool              0
BorrowerTotalMonthlyIncome         0
BorrowerOwnRent                    0
BorrowerCity                       2
BorrowerState                      2
IsCoBorrowerower                   0
CoBorrowerYearsInSchool            0
CoBorrowerAge                      0
CoBorrowerTotalMonthlyIncome       0
Approved                           0
TotalIncome                        0
E

In [277]:
# Assuming numerical_vars and categorical_vars are lists of numerical and categorical variables
numerical_vars = ['BorrowerAge', 'TotalLoanAmount', 'CreditScore', 'CLTV', 'DTI', 'BorrowerTotalMonthlyIncome']

categorical_vars = ['LoanPurpose','LoanType','ZipCode', 'BorrowerOwnRent', 'Education', 'LeadSourceGroup']

feats= numerical_vars + categorical_vars

#Drop below features
unused_vars = ['LeadID','DateAdded','LoanOfficer','Group','IsCoBorrowerower','BranchCode',
                    'CoBorrowerTotalMonthlyIncome','BorrowerYearsInSchool','CoBorrowerYearsInSchool','CoBorrowerAge','BorrowerCity',
                    'BorrowerState','Group','ApprovalDate']
loan_original.drop(unused_vars, axis=1, inplace=True)



In [278]:
loan_original.head(2)

,LeadSourceGroup,ZipCode,LoanNumber,LoanPurpose,LoanType,TotalLoanAmount,CreditScore,CLTV,DTI,BorrowerAge,BorrowerTotalMonthlyIncome,BorrowerOwnRent,Approved,TotalIncome,Education
0,Other,75,41032026021,Refinance,Conventional ...,206500.0,705,60.20,33.0,36,6083.00,Own,0,6083.00,PostGrad
1,TV,76,4103198104,Refinance,Conventional ...,100000.0,781,54.05,33.0,52,8883.71,Own,1,8883.71,UnderGrad


In [279]:
from sklearn.base import BaseEstimator, TransformerMixin

class Mapper(BaseEstimator, TransformerMixin):
    """Categorical variable mapper."""

    def __init__(self, variables: str, mappings: dict):

        if not isinstance(variables, str):
            raise ValueError("variables should be a str")

        self.variables = variables
        self.mappings = mappings

    def fit(self, X: pd.DataFrame, y: pd.Series = None):
        # we need the fit statement to accomodate the sklearn pipeline
        return self

    def transform(self, X: pd.DataFrame) -> pd.DataFrame:
        X = X.copy()
        #for feature in self.variables:
        X[self.variables] = X[self.variables].map(self.mappings).astype(int)

        return X

In [280]:
# map_loan_status=Mapper('Approved',{'No': 0, 'Yes': 1})
# for i in [map_loan_status]:
#   loan_original=i.fit(loan_original).transform(loan_original)

In [281]:
########## Split Features and Target Varible ############

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Split original data into train_df and test_df
train_df, test_df = train_test_split(loan_original, test_size=0.2, stratify=loan_original['Approved'], random_state=42)




In [282]:
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
import pandas as pd

class MissValImputer(BaseEstimator, TransformerMixin):
    """Custom imputer for handling missing values in numerical and categorical variables."""

    def __init__(self, numerical_vars: list, categorical_vars: list):
        if not all(isinstance(var, str) for var in numerical_vars + categorical_vars):
            raise ValueError("Variables should be a list of strings.")

        self.numerical_vars = numerical_vars
        self.categorical_vars = categorical_vars

    def fit(self, X: pd.DataFrame, y: pd.Series = None):
        # Store mean and standard deviation for numerical variables
        self.numerical_fill_values = {}

        for var in self.numerical_vars:
            mean_value = X[var].mean()
            std_value = X[var].std()
            self.numerical_fill_values[var] = (mean_value, std_value)

        # Store mode value for categorical variables
        self.categorical_fill_values = {}

        for var in self.categorical_vars:
            mode_value = X[var].mode().values[0]
            self.categorical_fill_values[var] = mode_value

        return self

    def transform(self, X: pd.DataFrame) -> pd.DataFrame:
        X = X.copy()

        np.random.seed(0)

        # Impute missing values for numerical variables with mean
        for var in self.numerical_vars:
            mean_value, std_value = self.numerical_fill_values[var]

            # Impute missing values using a random distribution
            null_count = X[var].isnull().sum()
            null_random_list = np.random.randint(mean_value - std_value, mean_value + std_value, size=null_count)
            X.loc[np.isnan(X[var]), var] = null_random_list
            X[var] = X[var].astype(X[var].dtype)  # Ensure the correct data type

        # Impute missing values for categorical variables with mode
        for var in self.categorical_vars:
            mode_value = self.categorical_fill_values[var]
            X[var].fillna(mode_value, inplace=True)

        return X


In [283]:
import pandas as pd

class OutlierHandler:
    def __init__(self, numerical_vars: list):
        self.numerical_vars = numerical_vars
        self.outlier_info = {}

    def identify_outliers(self, df, clmn):
        print('Outliers check for feature:', clmn)
        # Outliers check
        Q1 = df[clmn].quantile(0.25)
        Q3 = df[clmn].quantile(0.75)
        IQR = Q3 - Q1
        Lower_Whisker = Q1 - (1.5 * IQR)
        Upper_Whisker = Q3 + (1.5 * IQR)
        # print("Q1:", Q1, "Q3:", Q3, "IQR:", IQR, "Lower_Whisker:", Lower_Whisker, "Upper_Whisker:", Upper_Whisker,
        #       'Range:', (Upper_Whisker - Lower_Whisker))
        # Identify Outliers
        outliers_size = df[(df[clmn] < Lower_Whisker) | (df[clmn] > Upper_Whisker)].shape[0]
        dataset_size = df.shape[0]
        outliers_percentage = round((outliers_size / dataset_size) * 100, 2)
        # print('% of outliers:', outliers_percentage, '\n')

        self.outlier_info[clmn] = {'Lower_Whisker': Lower_Whisker, 'Upper_Whisker': Upper_Whisker}

    def treat_outliers(self, df, clmn, method='mean'):
        if method == 'mean':
            mean_value = round(df[clmn].mean(), 2)
            # print('Mean value for:', clmn, mean_value)
            df[clmn] = df[clmn].apply(lambda x: mean_value if
                                      (x < self.outlier_info[clmn]['Lower_Whisker'] or x > self.outlier_info[clmn]['Upper_Whisker'])
                                      else x)

    def fit(self, X, y=None):
        # Identify and store outlier information during fit
        for col in self.numerical_vars:
            self.identify_outliers(X, col)

        return self

    def transform(self, X):
        # Treat outliers during transform
        X = X.copy()
        for col in self.numerical_vars:
            self.treat_outliers(X, col)
        print("Outliers treatment is done")
        return X


In [284]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

class CategoricalEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, categorical_vars: list):
        self.categorical_vars = categorical_vars
        self.encoder = None

    def fit(self, X, y=None):
        # One-Hot Encoding
        encoder = OneHotEncoder(drop='first', sparse=False)
        self.encoder = encoder.fit(X[self.categorical_vars])

        return self

    def transform(self, X):
        if len(self.categorical_vars) == 0:
            return X

        # Prefix encoded values with column names
        OHE_df = pd.DataFrame(self.encoder.transform(X[self.categorical_vars]),
                              columns=self.encoder.get_feature_names_out(self.categorical_vars))

        num_df = X.drop(self.categorical_vars, axis=1).reset_index(drop=True)

        df_encoded = pd.concat([OHE_df, num_df], axis=1)

        return df_encoded


In [285]:
# import pickle

# pickle.dump(OHE, open('./encoder.pkl' ,'wb'))
# enc = pickle.load(open('./encoder.pkl','rb'))



In [286]:
# Split train_df into features (X_train) and target variable (y_train)
X_train = train_df.drop(columns='Approved')
y_train = train_df['Approved']

# Split test_df into features (X_test) and target variable (y_test)
X_test = test_df.drop(columns='Approved')
y_test = test_df['Approved']

# Optionally, convert to DataFrames if needed
X_train = pd.DataFrame(data=X_train, columns=X_train.columns)
X_test = pd.DataFrame(data=X_test, columns=X_test.columns)
y_train = pd.DataFrame(data=y_train, columns=['Approved'])
y_test = pd.DataFrame(data=y_test, columns=['Approved'])

In [287]:
X_train.head()

,LeadSourceGroup,ZipCode,LoanNumber,LoanPurpose,LoanType,TotalLoanAmount,CreditScore,CLTV,DTI,BorrowerAge,BorrowerTotalMonthlyIncome,BorrowerOwnRent,TotalIncome,Education
6210,Radio,77,4101207177,Refinance,Conventional ...,164000.0,675,80.00,44.0,41,5416.67,Own,9695.07,UnderGrad
1975,Radio,76,4103196243,Refinance,Conventional ...,158000.0,735,64.75,34.0,45,8166.67,Own,8166.67,UnderGrad
11897,Internet,76,41032023748,Refinance,Conventional ...,99000.0,709,60.00,45.0,56,4045.65,Own,4045.65,Higher School
7571,Other,76,41012010438,Refinance,Conventional ...,283500.0,717,88.59,46.0,44,2754.03,Own,7598.06,UnderGrad
10971,Radio,76,41052021392,Refinance,FHA ...,238451.0,664,96.49,27.0,30,6019.38,Rent,6019.38,Higher School


In [288]:
X_test.LoanPurpose.value_counts()

Refinance    2583
Name: LoanPurpose, dtype: int64

In [289]:
X_test.head()

,LeadSourceGroup,ZipCode,LoanNumber,LoanPurpose,LoanType,TotalLoanAmount,CreditScore,CLTV,DTI,BorrowerAge,BorrowerTotalMonthlyIncome,BorrowerOwnRent,TotalIncome,Education
5403,Internet,75,4102204361,Refinance,FHA ...,200313.0,670,87.49,41.0,55,3750.00,Own,8657.82,UnderGrad
8640,Other,76,41012014319,Refinance,Conventional ...,292000.0,746,80.00,17.0,51,11390.83,Own,16123.22,UnderGrad
5375,Internet,75,41012011401,Refinance,Conventional ...,140000.0,707,70.00,25.0,53,2000.00,Own,5389.53,UnderGrad
8138,Other,75,41052014817,Refinance,Conventional ...,227950.0,765,97.00,36.0,36,6416.72,Rent,6416.72,PostGrad
8467,TV,76,41052013475,Refinance,Conventional ...,322905.0,783,95.00,52.0,70,5209.08,Own,5209.08,PostGrad


In [290]:
y_train

,Approved
6210,1
1975,1
11897,1
7571,1
10971,0
...,...
9089,0
11082,1
12466,0
8829,1


In [291]:
from sklearn.pipeline import Pipeline

lac_pipe=Pipeline([
    ('miss_val_imputer', MissValImputer(numerical_vars, categorical_vars)),
    ('outlier_handler', OutlierHandler(numerical_vars)),
    ('categorical_encoder', CategoricalEncoder(categorical_vars)),


    # scale
    ('scaler', StandardScaler()),
    ('model_rf', RandomForestClassifier(n_estimators=150, max_depth=5,random_state=42))
])

lac_pipe.fit(X_train,y_train)
y_pred=lac_pipe.predict(X_test)

# Calculate the accuracy
print("Accuracy(in %):", accuracy_score(y_test, y_pred)*100)

Outliers check for feature: BorrowerAge
Outliers check for feature: TotalLoanAmount
Outliers check for feature: CreditScore
Outliers check for feature: CLTV
Outliers check for feature: DTI
Outliers check for feature: BorrowerTotalMonthlyIncome
Outliers treatment is done
Outliers treatment is done
Accuracy(in %): 72.28029423151374


In [292]:
# Persist the model
import joblib
joblib.dump(lac_pipe,'lac_pipe.joblib')

['lac_pipe.joblib']